In [11]:
import pickle

# Generate Column Names

In [12]:
import csv
import pandas as pd

In [13]:
column_names = ['gameId', 'creationTime', 'gameDuration', 'seasonId', 'winner', 'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald', 't1_champ1id', 't1_champ1_sum1', 't1_champ1_sum2', 't1_champ2id', 't1_champ2_sum1', 't1_champ2_sum2', 't1_champ3id', 't1_champ3_sum1', 't1_champ3_sum2', 't1_champ4id', 't1_champ4_sum1', 't1_champ4_sum2', 't1_champ5id', 't1_champ5_sum1', 't1_champ5_sum2', 't1_towerKills', 't1_inhibitorKills', 't1_baronKills', 't1_dragonKills', 't1_riftHeraldKills', 't1_ban1', 't1_ban2', 't1_ban3', 't1_ban4', 't1_ban5', 't2_champ1id', 't2_champ1_sum1', 't2_champ1_sum2', 't2_champ2id', 't2_champ2_sum1', 't2_champ2_sum2', 't2_champ3id', 't2_champ3_sum1', 't2_champ3_sum2', 't2_champ4id', 't2_champ4_sum1', 't2_champ4_sum2', 't2_champ5id', 't2_champ5_sum1', 't2_champ5_sum2', 't2_towerKills', 't2_inhibitorKills', 't2_baronKills', 't2_dragonKills', 't2_riftHeraldKills', 't2_ban1', 't2_ban2', 't2_ban3', 't2_ban4', 't2_ban5']

In [14]:
print(len(column_names))

61


# Helper Functions

In [15]:
def get_first_blood_team(info):
    '''
    info: the detailed information for two teams in a single match.
    Return the team number that get the first blood in a single match. Return 0 if no such team exists.
    '''
    for participant in info['participants']:
        if participant["firstBloodKill"]:
            # Find the first blood then return the team number.
            # The return value matches the teams' order in info['teams']
            if participant['teamId'] == info['teams'][0]['teamId']:
                return 1
            else:
                return 2
    # No first blood in current match.
    return 0

In [16]:
def get_first_tower_team(info):
    '''
    info: the detailed information for two teams in a single match.
    Return the team which destroy the first tower in a single match. If no team destroies the inhibitor, then return 0.
    '''
    for participant in info['participants']:
        if participant["firstTowerKill"]:
            # Find the first tower then return the team number.
            # The return value matches the teams' order in info['teams']
            if participant['teamId'] == info['teams'][0]['teamId']:
                return 1
            else:
                return 2
    # No first blood in current match.
    return 0

In [17]:
def get_first_inhibitor_team(info):
    '''
    info: the detailed information for two teams in a single match.
    Return the team which destroy the first inhibitor in a single match. If no team destroies the inhibitor, then return 0.
    '''
    if info['teams'][0]['objectives']['inhibitor']['first']:
        return 1
    elif info['teams'][1]['objectives']['inhibitor']['first']:
        return 2
    else:
        return 0

In [18]:
def get_first_baron_team(info):
    '''
    info: the detailed information for two teams in a single match.
    Return the team which kill the first baron in a single match. If no team kills the baron, then return 0.
    '''
    if info['teams'][0]['objectives']['baron']['first']:
        return 1
    elif info['teams'][1]['objectives']['baron']['first']:
        return 2
    else:
        return 0

In [19]:
def get_first_dragon_team(info):
    '''
    info: the detailed information for two teams in a single match.
    Return the team which kill the first dragon in a single match. If no team kills the dragon, then return 0.
    '''
    if info['teams'][0]['objectives']['dragon']['first']:
        return 1
    elif info['teams'][1]['objectives']['dragon']['first']:
        return 2
    else:
        return 0

In [20]:
def get_first_rift_herald_team(info):
    '''
    info: the detailed information for two teams in a single match.
    Return the team which kill the first rift herald in a single match. If no team kills the rift_herald, then return 0.
    '''
    if info['teams'][0]['objectives']['riftHerald']['first']:
        return 1
    elif info['teams'][1]['objectives']['riftHerald']['first']:
        return 2
    else:
        return 0

In [21]:
def create_csv_for_one_day(extracted_pkl, df):
    new_rows = []
    for match in extracted_pkl:

        # Remove repetitive record by filtering game id.
        if match['info']['gameId'] in df['gameId'].unique():
            continue
        
        # Filter out records with invalid number of teams.
        if len(match['info']['teams']) != 2:
            continue

        # Find the winner for current match
        if match['info']['teams'][0]['win']:
            winner = 1
        else:
            winner = 2

        new_row = {
            'gameId': match['info']['gameId'],
            'creationTime': match['info']['gameCreation'],
            'gameDuration': match['info']['gameDuration'],
            'seasonId': 13,
            'winner': winner,
            'firstBlood': get_first_blood_team(match['info']),
            'firstTower': get_first_tower_team(match['info']),
            'firstInhibitor': get_first_inhibitor_team(match['info']),
            'firstBaron': get_first_baron_team(match['info']),
            'firstDragon': get_first_dragon_team(match['info']),
            'firstRiftHerald': get_first_rift_herald_team(match['info']),
        }
        
        # Create champion and summoner spells columns -- 30 columns in total
        team_participant_count = {match['info']['teams'][0]['teamId']: 0, match['info']['teams'][1]['teamId']: 0}
        for participant in match['info']['participants']:
            team_id = 't1_' if participant['teamId'] == match['info']['teams'][0]['teamId'] else 't2_'
            
            team_participant_count[participant['teamId']] += 1
            player_num = team_participant_count[participant['teamId']]
            
            # Champion and summoner spells
            new_row[f'{team_id}champ{player_num}id'] = participant['championId']
            new_row[f'{team_id}champ{player_num}_sum1'] = participant['summoner1Id']
            new_row[f'{team_id}champ{player_num}_sum2'] = participant['summoner2Id']
        
        # print(len(match['info']['teams'][0]['bans']))
        if len(match['info']['teams'][0]['bans']) != 5 and len(match['info']['teams'][0]['bans']) != 5:
            # Skip the matches with invalid bans array
            continue
        
        # Create objectives columns(5 columns) and bans columns(5 columns) for two teams -- 10 columns for each team
        for index, team in enumerate(match['info']['teams']):
            team_id = f't{index + 1}'
            
            # objective columns
            new_row[f'{team_id}_towerKills'] = team['objectives']['tower']['kills']
            new_row[f'{team_id}_inhibitorKills'] = team['objectives']['inhibitor']['kills']
            new_row[f'{team_id}_baronKills'] = team['objectives']['baron']['kills']
            new_row[f'{team_id}_dragonKills'] = team['objectives']['dragon']['kills']
            new_row[f'{team_id}_riftHeraldKills'] = team['objectives']['riftHerald']['kills']
            
            # bans columns
            if (len(team['bans']) == 5):
                sorted_bans = sorted(team['bans'], key=lambda x: x['pickTurn'])

                team_ban_count = 1
                for ban in sorted_bans:
                    new_row[f't{index + 1}_ban{team_ban_count}'] = int(ban['championId'])
                    team_ban_count += 1

        new_rows.append(new_row)
        
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    print(len(df))
    return df


# Creating CSV

In [26]:
df = pd.DataFrame(columns=column_names)
df

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5


## Nov 15

In [27]:
extracted = []
with open('/Users/wen/Desktop/match_info.pkl', 'rb') as file:
    extracted = pickle.load(file)

In [28]:
df = create_csv_for_one_day(extracted, df)

3582


## Nov 16

In [30]:
extracted_16 = []
with open('/Users/wen/Desktop/match_info1.7k_Nov16.pkl', 'rb') as file:
    extracted_16 = pickle.load(file)
len(extracted_16)

1701

In [31]:
df = create_csv_for_one_day(extracted_16, df)

4801


### Nov 17

In [32]:
extracted_17 = []
with open('/Users/wen/Desktop/match_info12.9k_Nov17.pkl', 'rb') as file:
    extracted_17 = pickle.load(file)
len(extracted_17)

12901

In [33]:
df = create_csv_for_one_day(extracted_17, df)

14147


### Nov 18

In [34]:
extracted_18 = []
with open('/Users/wen/Desktop/match_info38k_Nov18.pkl', 'rb') as file:
    extracted_18 = pickle.load(file)
len(extracted_18)

38044

In [35]:
df = create_csv_for_one_day(extracted_18, df)

39246


### Nov 19

In [36]:
extracted_19 = []
with open('/Users/wen/Desktop/FLEXSR_matche_info_save14k_Nov19.pkl', 'rb') as file:
    extracted_19 = pickle.load(file)
len(extracted_19)

14763

In [37]:
df = create_csv_for_one_day(extracted_19, df)

46968


### Nov 20

In [38]:
extracted_20 = []
with open('/Users/wen/Desktop/match_info41k_Nov20.pkl', 'rb') as file:
    extracted_20 = pickle.load(file)
len(extracted_20)

41819

In [39]:
df = create_csv_for_one_day(extracted_20, df)

59719


### Nov 22

In [40]:
extracted_22 = []
with open('/Users/wen/Desktop/FLEXSR_matche_info_save14k_Nov22.pkl', 'rb') as file:
    extracted_22 = pickle.load(file)
len(extracted_22)

14564

In [41]:
df = create_csv_for_one_day(extracted_22, df)

62611


# Nov 23

In [42]:
extracted_23 = []
with open('/Users/wen/Desktop/match_info32k_Nov23.pkl', 'rb') as file:
    extracted_23 = pickle.load(file)
len(extracted_23)

32569

In [43]:
df = create_csv_for_one_day(extracted_23, df)

70105


# Nov 27

In [44]:
extracted_27 = []
with open('/Users/wen/Desktop/match_info25k_Nov27.pkl', 'rb') as file:
    extracted_27 = pickle.load(file)
len(extracted_27)

25321

In [45]:
df = create_csv_for_one_day(extracted_27, df)

80831


# Dec 7

In [46]:
extracted_12_7 = []
with open('/Users/wen/Desktop/match_info18k_Dec7.pkl', 'rb') as file:
    extracted_12_7 = pickle.load(file)
len(extracted_12_7)

18618

In [47]:
df = create_csv_for_one_day(extracted_12_7, df)

92464


# Dec 8

In [48]:
extracted_12_8 = []
with open('/Users/wen/Desktop/match_info50k_Dec8.pkl', 'rb') as file:
    extracted_12_8 = pickle.load(file)
len(extracted_12_8)

50864

In [49]:
df = create_csv_for_one_day(extracted_12_8, df)

112809


# Save Final CSV

In [50]:
df.to_csv('/Users/wen/Desktop/match_data.csv', index=False)